In [ ]:
!git clone <GITHUB_API_KEY>/jpradov/ipeo_project.git
!pip install kornia imagecodecs==2023.9.18 wandb

In [ ]:
from google.colab import drive
import os
import sys
sys.path.append('/content/ipeo_project')
sys.path.append('/content/ipeo_project/utils')

drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from utils import data
import pandas as pd
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
import config
from tqdm import tqdm
import kornia
import matplotlib.pyplot as plt
import numpy as np
from utils import evaluation

from models.msnet import MSNet
from torch.utils.data import DataLoader
from utils.data import create_dataloaders
from utils.train import run_training


In [ ]:
# Set seeds for reproducibility

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

In [ ]:
#Should be cuda:0 in colab and cpu in local.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#DATA_PATH = config.PATH_TO_DATA
DATA_PATH = "/content/drive/MyDrive/ipeo_project/dataset/"

In [ ]:
from models.deeplabv3plus import fetch_deeplabv3
%load_ext autoreload
%autoreload 2

max_epochs = 5
lrs = [1e-3, 1e-4, 1e-5]
for learning_rate in lrs:
  schedulers = ["CosALRScheduler", "noScheduler"]

  for scheduler_name in schedulers:
    model = fetch_deeplabv3(output_stride=8, num_classes=2).to(device)

    optimizer = optim.AdamW(model.parameters(),lr=learning_rate)
    if scheduler_name == "CosALRScheduler" :
          scheduler = CosineAnnealingLR(optimizer,max_epochs)
    elif scheduler_name == "noScheduler":
          scheduler = None

    criterion = nn.CrossEntropyLoss()
    results = run_training(experiment_name=f"DeepLabv3plus_lr_{learning_rate}_{scheduler_name}_AdamWOpt",
                           model=model,num_epochs=max_epochs,
                           lr=learning_rate,
                           batch_size=32,
                           device=device,
                           data_dir=DATA_PATH,
                           bands=[0,1,2],
                           scheduler=scheduler,
                           optimizer=optimizer,
                           project_name="DeepLabv3plus_baseline")

In [ ]:
from utils.evaluation import evaluate

train_dl, val_dl, test_dl = create_dataloaders(
        data_dir=DATA_PATH, batch_size=32, bands=[0,1,2], num_workers=0)

result = evaluate(model, device, test_dl, criterion)
print(result)